In [3]:
from flask import Flask, request, jsonify
from ultralytics import YOLO
import torch
from PIL import Image
import io

# Inisialisasi aplikasi Flask
app = Flask(__name__)

# Load model YOLOv8 yang telah dilatih
model = YOLO('yolov8n_custom_trained.pt')

# Endpoint untuk prediksi
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    # Ambil file gambar dari request
    file = request.files['image']
    img_bytes = file.read()
    img = Image.open(io.BytesIO(img_bytes))

    # Lakukan prediksi menggunakan model
    results = model(img)

    # Ekstraksi hasil prediksi
    predictions = []
    for result in results:
        for box in result.boxes:
            predictions.append({
                'class': box.cls.item(),
                'confidence': box.conf.item(),
                'box': box.xyxy.tolist()
            })

    # Kembalikan hasil prediksi sebagai JSON
    return jsonify(predictions)

# Jalankan aplikasi
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.26:5000
Press CTRL+C to quit
192.168.1.26 - - [26/Aug/2024 15:36:39] "GET / HTTP/1.1" 404 -
192.168.1.26 - - [26/Aug/2024 15:36:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2024 15:36:49] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2024 15:36:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2024 15:36:58] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2024 15:36:58] "GET /favicon.ico HTTP/1.1" 404 -


In [2]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [4]:
from flask import Flask, request, render_template, redirect, url_for
from ultralytics import YOLO
from PIL import Image
import io

app = Flask(__name__)

# Inisialisasi model YOLO
model = YOLO('yolov8n_custom_trained.pt')  # Pastikan model ini sudah diunduh

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Periksa apakah ada file yang diunggah
        if 'file' not in request.files:
            return redirect(request.url)
        
        file = request.files['file']
        
        # Jika tidak ada file yang dipilih
        if file.filename == '':
            return redirect(request.url)
        
        if file:
            # Baca gambar dan lakukan prediksi
            image = Image.open(file.stream)
            results = model(image)

            # Render hasil prediksi
            return render_template('result.html', results=results)

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Axioo Pongo\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
from ultralytics import YOLO
import cv2

# Load the pre-trained YOLO model
model = YOLO('yolov8n_custom_trained.pt')  # Ganti dengan path ke model Anda

# Read the input image
image_path = 'kayulog.jpg'  # Ganti dengan path ke gambar Anda
image = cv2.imread(image_path)

# Perform prediction
results = model(image)

# Process and display the results
for result in results:
    # Extract boxes, confidences, and class IDs
    boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
    confidences = result.boxes.conf.cpu().numpy()  # Get confidences
    class_ids = result.boxes.cls.cpu().numpy()  # Get class IDs

    for box, confidence, class_id in zip(boxes, confidences, class_ids):
        # Draw bounding box and label on the image
        x1, y1, x2, y2 = map(int, box)
        label = f'{model.names[int(class_id)]}: {confidence:.2f}'
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Show the result
cv2.imshow('Predictions', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 448x640 106 woods, 50.7ms
Speed: 6.0ms preprocess, 50.7ms inference, 144.5ms postprocess per image at shape (1, 3, 448, 640)


In [6]:
from ultralytics import YOLO
import cv2

# Load the pre-trained YOLO model
model = YOLO('yolov8n_custom_trained.pt')  # Ganti dengan path ke model Anda

# Read the input image
image_path = 'kayuitung.jpg'  # Ganti dengan path ke gambar Anda
image = cv2.imread(image_path)

# Perform prediction
results = model(image)

# Process and display the results
object_count = 0  # Counter for detected objects

for result in results:
    # Extract boxes, confidences, and class IDs
    boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes

    for box in boxes:
        object_count += 1
        x1, y1, _, _ = map(int, box)
        # Draw the number on the image
        cv2.putText(image, str(object_count), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Write the total number of detected objects on the image
total_text = f'Total : {object_count}'
cv2.putText(image, total_text, (10, image.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Show the result
cv2.imshow('Predictions', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x512 (no detections), 64.4ms
Speed: 5.0ms preprocess, 64.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 512)


In [5]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load model YOLOv8
model = YOLO('yolov8n.pt')  # Ganti dengan model yang sesuai

# Kalibrasi: Lebar objek yang diketahui dalam cm dan dalam piksel
known_width_cm = 7.5  # Lebar objek yang diketahui dalam cm (misal penggaris)
known_width_pixels = 311  # Lebar objek dalam piksel (misalnya dari hasil sebelumnya)

# Hitung skala konversi dari piksel ke cm
pixel_to_cm_ratio = known_width_cm / known_width_pixels

# Fungsi untuk mengukur panjang dan lebar objek dalam cm
def measure_object(image_path):
    # Baca gambar dari file
    frame = cv2.imread(image_path)
    
    # Lakukan deteksi objek
    results = model(frame)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Koordinat bounding box dalam integer
            length_pixels = x2 - x1
            width_pixels = y2 - y1

            # Konversi ke cm menggunakan skala yang telah dihitung
            length_cm = length_pixels * pixel_to_cm_ratio
            width_cm = width_pixels * pixel_to_cm_ratio

            # Gambar bounding box dan ukuran dalam cm pada gambar
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'L: {length_cm:.2f} cm, W: {width_cm:.2f} cm', (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            print(f"Panjang: {length_cm:.2f} cm, Lebar: {width_cm:.2f} cm")

    # Simpan dan tampilkan hasil
    output_image_path = 'output_image.jpg'
    cv2.imwrite(output_image_path, frame)
    cv2.imshow('Deteksi Objek dan Pengukuran', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Gambar yang akan digunakan
image_path = 'gelas_9_30.jpg'  # Ganti dengan path ke file gambar Anda
measure_object(image_path)


0: 640x480 1 cup, 28.5ms
Speed: 8.1ms preprocess, 28.5ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 480)
Panjang: 7.50 cm, Lebar: 10.76 cm


In [4]:
import cv2
from ultralytics import YOLO

# Load model YOLOv8
model = YOLO('yolov8n.pt')  # Ganti dengan model yang sesuai

# Fungsi untuk menghitung lebar objek dalam piksel
def calculate_object_width_in_pixels(image_path):
    # Baca gambar dari file
    frame = cv2.imread(image_path)
    
    # Lakukan deteksi objek
    results = model(frame)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Koordinat bounding box dalam integer
            width_pixels = x2 - x1  # Menghitung lebar dalam piksel

            print(f"Lebar objek: {width_pixels} piksel")

            # Gambar bounding box pada gambar
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'W: {width_pixels} px', (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Tampilkan gambar
    cv2.imshow('Deteksi Objek', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Gambar yang akan digunakan
image_path = 'gelas_9_30.jpg'  # Ganti dengan path ke file gambar Anda
calculate_object_width_in_pixels(image_path)



0: 640x480 1 cup, 98.8ms
Speed: 18.5ms preprocess, 98.8ms inference, 6.3ms postprocess per image at shape (1, 3, 640, 480)
Lebar objek: 311 piksel
